In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Input,Flatten,Reshape,AveragePooling2D,Dropout,LayerNormalization, ReLU,concatenate,Cropping2D, BatchNormalization
from keras.layers import Conv2D, Conv2DTranspose, Input,Dropout, ReLU,BatchNormalization,Concatenate,LeakyReLU,Identity
import tensorflow as tf

import keras
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
from PIL import Image, ImageDraw, ImageFont
font_path = '../dados/targa/Targa.ttf'

In [ ]:
!wget  -nc https://www.dropbox.com/scl/fi/uaiyxp0t2l8hfcszfadtj/dados.zip?rlkey=lnqcb79vbu8j6cdbfgofogius&dl=1
!unzip -n -q dados.zip?rlkey=lnqcb79vbu8j6cdbfgofogius

In [ ]:
image_path = '../dados/CAPTCHA-10k/treinamento'
def generate_df(image_path):
  label_path = '../dados/CAPTCHA-10k/labels10k'

  jpg_files = [f for f in os.listdir(image_path) if f.endswith('.jpg')]
  jpg_files.sort()
  data = []

  for jpg_file in jpg_files:
      txt_file = os.path.splitext(jpg_file)[0] + '.txt'
      txt_file_path = os.path.join(label_path, txt_file)

      if os.path.exists(txt_file_path):
          with open(txt_file_path, 'r') as file:
              txt_content = file.read().strip()

          data.append({'jpg_file': jpg_file, 'txt_content': txt_content})
  return pd.DataFrame(data)

df = generate_df(image_path)
df.head()

In [ ]:
vocab = np.unique(list(df['txt_content'].sum()))
vocab = list(vocab)
np.array(vocab)

In [ ]:
def generate_clean_captcha(text):
    # Fixed parameters
    size = (180, 50)  # Change size to (height, width)
    font_size = 24
    num_parts = 6

    # Create a blank white image
    image = Image.new('L', size, 255)  # 'L' mode for grayscale

    # Load the custom font
    font = ImageFont.truetype(font_path, font_size)

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Calculate positions for each part
    part_width = size[0] / num_parts
    horizontal_positions = [int(part_width * i + part_width / 2) for i in range(num_parts)]
    horizontal_positions = horizontal_positions[:len(text)]  # Adjust to the length of the text

    # Calculate y position to center the text vertically
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_height = text_bbox[3] - text_bbox[1]
    text_y = (size[1] - text_height) // 2

    # Draw each letter at the calculated position
    for char, x in zip(text, horizontal_positions):
        char_bbox = draw.textbbox((0, 0), char, font=font)
        char_width = char_bbox[2] - char_bbox[0]
        char_x = x - char_width // 2  # Center the character horizontally within its part
        draw.text((char_x, text_y), char, font=font, fill=0)

    # Convert to numpy array if needed for further processing with OpenCV
    captcha_image = np.array(image)

    return captcha_image

In [ ]:
def generate_X_Y(image_path):
  df = generate_df(image_path)

  num_classes = len(vocab)
  char_to_index = {char: idx for idx, char in enumerate(vocab)}
  X = []
  Y = []

  for text in df['txt_content']:
    x = generate_clean_captcha(text[:6])
    interval = [0,30,60,90,120,150,180]
    for i in range(len(interval)-1):
        fake_img = x[:,interval[i]:interval[i+1]]
        y = np.zeros((1,num_classes))
        y[0,char_to_index[text[i]]] = 1
        X.append(fake_img)
        Y.append(y)

  X = np.array(X)
  Y = np.array(Y)

  X = (X.astype('float32') - 127.5)  / 127.5

  X = X.reshape(-1,50,30,1)
  Y = Y.reshape(-1,num_classes)
  return X,Y

X,Y = generate_X_Y('../dados/CAPTCHA-10k/treinamento')

In [ ]:
plt.imshow(X[0])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

datagen.fit(X)
batch_size = 8
augmented_data_generator = datagen.flow(X, Y, batch_size=batch_size)

In [ ]:
#Camada Convolution-BatchNorm-ReLu
def CK(filters, kernel_size=(4, 4), strides=(2, 2), padding='same', use_batch_norm=True, downsample=True):
    '''
        filters: quantidade de filtros
        kernel_size 3x3 | strides 1x1 | padding same | sao constantes durante o codigo
        use_batch_norm ->   indica quando devemos usar BatchNormalization, em caso de negativo, a camada se torna a Identidade
        downsample ->       indica se a dimensao deve aumentar ou diminuir 
    '''

    # Esse chavemento usando  if ternario serve para selecionar as camadas com base nos atributos
    conv = Conv2D               if downsample       else Conv2DTranspose
    norm = BatchNormalization   if use_batch_norm   else Identity
    actf = LeakyReLU(0.2)       if downsample       else ReLU()
    # alpha de 0.2 na LeakyReLU foi definido no paper original

    # Com o chaveamento pronto, a camada pode ser montada sequencialmente
    def layer(x):
        x = conv(filters, kernel_size, strides=strides, padding=padding)(x)
        x = norm()(x)
        x = actf(x)
        return x
    return layer

In [ ]:
def discriminator(output_nc, ngf, num_downsample=3):

    # O discriminador por sua vez, recebe a entrada e a saida do modelo, tentando assim decidir se aquilo 'e real ou nao
    tar = Input(shape=[50, 30, output_nc], name='target_image')

    # Initial convolutional layers # SEM BATCH NORM !
    x = CK(ngf,use_batch_norm=False)(tar)

    # Contracting path
    for i in range(num_downsample):
        x = CK(ngf*(2**(i+1)))(x)
    
    # Por fim, a patchGan gera uma classificao binaria por patch, o tamanho do patch eh definido pelo num_downsample. Quanto maior, menor a area de recepcao
    # Por exemplo num_downsample = 4 faz com que o discriminador classifique blocos de 16x16
    x = Conv2D(ngf//2, (1, 1),activation='ReLU')(x)
    x = Flatten()(x)
    # x = Dense(64,activation="ReLU")(x)
    x = Dense(37,activation="softmax")(x)
    return Model(inputs=tar, outputs=x)

classifier = discriminator(1,64,2)
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

In [ ]:
class StopTrainingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') == 1.0:
            print("\nReached 100% accuracy, stopping training!")
            self.model.stop_training = True
        
        if logs.get('val_accuracy') == 1.0:
            print("\nReached 100% accuracy on val, stopping training!")
            self.model.stop_training = True

checkpoint = ModelCheckpoint(
    'classifier_pre_trained_best.tf',
    monitor='val_accuracy',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [ ]:
stop_training_callback = StopTrainingCallback()
classifier.fit(augmented_data_generator,steps_per_epoch=len(X) // batch_size, epochs=500, 
               callbacks=[stop_training_callback,checkpoint],
               validation_data=(X, Y))

In [ ]:
classifier.save("classifier_pre_trained.tf")

In [ ]:
classifier.evaluate(X,Y)